In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  GridSearchCV, KFold,StratifiedKFold

In [3]:
train_data = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\train.csv')
x_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\test.csv')
x_train = train_data.drop(columns='TARGET')
y_train = train_data['TARGET'].astype('float')


In [4]:
x_test = x_test.set_index('SK_ID_CURR')
x_train = x_train.set_index('SK_ID_CURR')
submit = pd.DataFrame(x_test.index)

### Predict

In [5]:
# model = LogisticRegression(penalty='elasticnet', l1_ratio=0.8, solver='saga')
# model.fit(x_train,y_train)
# y_pred = model.predict_proba(x_test)[:, 1]


In [6]:
kfold = KFold(n_splits=5, shuffle=True, random_state= 1710)
strkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state= 1710)

In [7]:

# Define the logistic regression model
logreg = LogisticRegression()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.001, 0.01, 0.05, 1, 10, 100],
    'penalty': ['l2'],  
    'solver': ['liblinear'],
    'max_iter': [100,300,500]
}

# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(logreg, param_grid, cv=kfold, scoring='roc_auc', verbose= 10)
grid_search.fit(x_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5; 1/18] START C=0.001, max_iter=100, penalty=l2, solver=liblinear........


[CV 1/5; 1/18] END C=0.001, max_iter=100, penalty=l2, solver=liblinear;, score=0.743 total time=  11.6s
[CV 2/5; 1/18] START C=0.001, max_iter=100, penalty=l2, solver=liblinear........
[CV 2/5; 1/18] END C=0.001, max_iter=100, penalty=l2, solver=liblinear;, score=0.754 total time=   8.8s
[CV 3/5; 1/18] START C=0.001, max_iter=100, penalty=l2, solver=liblinear........
[CV 3/5; 1/18] END C=0.001, max_iter=100, penalty=l2, solver=liblinear;, score=0.751 total time=   8.1s
[CV 4/5; 1/18] START C=0.001, max_iter=100, penalty=l2, solver=liblinear........
[CV 4/5; 1/18] END C=0.001, max_iter=100, penalty=l2, solver=liblinear;, score=0.749 total time=   7.4s
[CV 5/5; 1/18] START C=0.001, max_iter=100, penalty=l2, solver=liblinear........
[CV 5/5; 1/18] END C=0.001, max_iter=100, penalty=l2, solver=liblinear;, score=0.746 total time=   7.3s
[CV 1/5; 2/18] START C=0.001, max_iter=300, penalty=l2, solver=liblinear........
[CV 1/5; 2/18] END C=0.001, max_iter=300, penalty=l2, solver=liblinear;, sc

In [8]:
# Make predictions on the test set using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict_proba(x_test)[:, 1]

In [11]:
x_test.index

Int64Index([ 83659, 174814, 179486,  57038,  25672, 302672,  90463, 178370,
              7865, 185100,
            ...
             36512,  17179,  34093,  57005,  68015, 150442,   5217, 260741,
            284794, 258643],
           dtype='int64', name='SK_ID_CURR', length=61502)

In [9]:
submit['TARGET'] = y_pred
submit = submit.set_index('SK_ID_CURR').sort_index(ascending=True)
submit.to_csv('../Submission.csv')


In [15]:
submit

,TARGET
SK_ID_CURR,
5,0.019868
13,0.118634
16,0.061751
19,0.175175
25,0.083682
...,...
307487,0.023248
307490,0.017388
307497,0.031494
